In [6]:
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

import os
import yaml
from crewai import Agent, Task, Crew

os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

In [7]:
# Define file paths for YAML configurations
files = {
    'agents': 'agents.yaml',
    'tasks': 'tasks.yaml'
}

In [8]:
# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [9]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

# Creating Agents
resource_manager = Agent(
  config=agents_config['resource_manager']
)

maintenance_supervisor = Agent(
  config=agents_config['maintenance_supervisor']
)

health_monitor = Agent(
  config=agents_config['health_monitor']
)

policy_manager = Agent(
    config=agents_config['policy_manager']
    )


In [10]:
# Creating Tasks
resource_optimization = Task(
  config=tasks_config['resource_optimization'],
  agent=resource_manager
)

maintenance_routine = Task(
  config=tasks_config['equipment_maintenance'],
  agent=maintenance_supervisor
)

health_coordination = Task(
  config=tasks_config['health_analysis'],
  agent=health_monitor,
  output_pydantic=ProjectPlan # This is the structured output we want
)

policy_decision = Task(
    config=tasks_config['policy_decision'],
    agent=policy_manager,
    output_pydantic=ProjectPlan # This is the structured output we want

)